In [1]:
import sqlalchemy
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from datetime import datetime
from sqlalchemy import Column, text, create_engine
from sqlalchemy.types import TIMESTAMP
from sqlalchemy.dialects.mysql import BIGINT
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import glob

In [2]:
path="C:\\Users\\makol\\Documents\\Python_Scripts\\Git\\user_scripts\\masha_k\\Analyze_validation\\ngpt\\"

In [3]:
df_prom=pd.read_csv(path+'2020.08.07.csv',encoding='windows-1251',sep=',')

C:\Users\makol\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_prom.head()

,Дата и время поездки,Терминал,Номер билета,Номер кристалла,TCAT,NUM_TRIP,Код Маршрута,Табельный номер,Код предприятия
0,2020-08-07 05:51:34,81129629,18,5D658236,ЕТК,0,11303456789,955,5
1,2020-08-07 05:52:49,81129629,23,B7C3090E,ЕТК,0,11303456789,955,5
2,2020-08-07 05:55:42,81129629,31,2DBE8DC1,ЕТК,0,11303456789,955,5
3,2020-08-07 05:59:20,81129629,38,C18EE623,ЕТК,0,11303456789,955,5
4,2020-08-07 06:06:34,81129629,48,910FDC23,ЕТК,0,11303456789,955,5


In [5]:
df_prom['count']=1

In [6]:
df_prom2=pd.DataFrame(df_prom.groupby(['TCAT']).agg({'count': np.sum})).reset_index()

In [7]:
df_prom2

,TCAT,count
0,БАНК,89235
1,БЕТК,6309
2,БЕТК ТЛТ,525
3,БКС,3791
4,БКШ,1484
5,БКШ НК,4
6,ЕТК,21078
7,КС,5666
8,КС(льг),44
9,КШ,6629


In [8]:
df=pd.DataFrame()

In [9]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://makolesnikova@infraeconomy.com:zahceelaS4aewie7ouyohjoFuphah9Gu@84.201.144.84:54327/cei_samara')

In [10]:
meta = MetaData()
tableval = Table('validations_validationngptwithbank', meta, autoload=True, autoload_with=engine)
conn = engine.connect()
Session = sessionmaker(bind = engine)
session = Session()

In [11]:
for i,f in enumerate(glob.iglob(path+'2020.08*.csv')):
    df_prom=(pd.read_csv(f,encoding='windows-1251',sep=','))
    #df_prom2=pd.DataFrame(df_prom.groupby(['Код Маршрута']).agg({'count': np.sum})).reset_index()
    df=df.append(df_prom)

C:\Users\makol\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\makol\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\makol\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
len(df)

9855419

In [13]:
df.columns = ['trip_datetime', 'terminal_id', 'ticket_num', 'crystal_id', 'tcat', 'num_trip', 'val_marsh_id', 'table_num',' carrier_code']

In [14]:
df=df.tail(len(df)-)

,trip_datetime,terminal_id,ticket_num,crystal_id,tcat,num_trip,val_marsh_id,table_num,carrier_code
0,2020-08-01 06:54:02,81129566,13,E4A5CB2D,ЕТК,0,10103456789,989,5
1,2020-08-01 07:32:25,81129566,19,A2F1FB36,КС,0,10103456789,989,5
2,2020-08-01 08:12:16,81129566,44,3E036E36,КС,0,10103456789,989,5
3,2020-08-01 09:03:03,81129566,76,7DF2E2C1,КС,0,10103456789,989,5
4,2020-08-01 09:05:33,81129566,77,7D129DD0,КС,0,10103456789,989,5


In [15]:
cols = ",".join([str(i) for i in df.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df.iterrows():
    sql = "INSERT INTO \"validations_validationngptwithbank\" (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    conn.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    session.commit()